# Entrega 4 SM

In [10]:
import os

## Primeros pasos: Cargar el archivo de audio (sonido estéreo vs mono)



In [19]:
audio_input_path="Prueba"
print(audio_input_path)
print(type(audio_input_path))

Prueba
<class 'str'>


In [17]:

filename = os.path.join(audio_input_path, 'sample1_stereo.wav')
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')


NameError: name 'wavfile' is not defined